In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.table import Table, join, QTable
import astropy.units as u
import sys
import pyneb as pn
from astropy.io import fits
from matplotlib import gridspec
from astropy.io import ascii 
import scipy
from scipy.optimize import curve_fit
from scipy.integrate import quad
import numpy.linalg as la
import multiprocessing as mp
import math
import extinction
from extinction import apply, remove
from orb.fit import fit_lines_in_spectrum
from orb.utils.spectrum import corr2theta, amp_ratio_from_flux_ratio
from orb.core import Lines
from orcs.process import SpectralCube
import gvar
import orb

### Import Spectra Data and PHANGS-MUSE Nebular Catalog for selected galaxy

In [ ]:
galdic = {1:'NGC4254', 2:'NGC4535', 3:'NGC3351', 4:'NGC2835', 5:'NGC0628'}
galaxy = galdic[4]
galveldic = {'NGC4254': 2388 , 'NGC4535': 1954  , 'NGC3351': 775, 'NGC2835': 867, 'NGC0628':651}
galvel = galveldic[galaxy]
galebv = {'NGC4254': 0.0334 , 'NGC4535': 0.0168 , 'NGC3351': 0.0239, 'NGC2835': 0.0859, 'NGC0628': 0.0607}
ebv = galebv[galaxy]

hdu1 = fits.open(f'/home/habjan/jupfiles/data/{galaxy}_VorSpectra.fits')
hdu2 = fits.open(f'/home/habjan/jupfiles/data/{galaxy}_ppxf-bestfit-emlines.fits')

bestfit_gas = np.ma.MaskedArray( hdu2[1].data["BESTFIT"],
                                mask=hdu2[1].data['BESTFIT']==0)

gas_templ = np.ma.MaskedArray( hdu2[1].data["GAS_BESTFIT"],
                              mask=hdu2[1].data['BESTFIT']==0)

lam = np.exp(hdu1[2].data['LOGLAM'])
log_spec = hdu1[1].data['SPEC']

infile = open("/home/habjan/jupfiles/data/Nebulae_catalogue_v3.fits",'rb')
hdul = Table.read(infile)
data = hdul[hdul['gal_name'] == f'{galaxy}']

infile = f"/home/habjan/jupfiles/data/{galaxy}_cube.hdf5"
cube = SpectralCube(infile)

hdul = fits.open(f'/home/habjan/jupfiles/data/{galaxy}_SITELLE_Spectra.fits')
sitlam = hdul[0].data
sitspec = hdul[1].data - hdul[2].data

galaxy

### Chose a S/N value to cut values

In [ ]:
sn = 3
fluxin = log_spec - (bestfit_gas - gas_templ)
mciters = 10**3

### [OII]7319 Error Function

In [ ]:
def mcerr7319(inwave, inputflux, gnoise, wave0, iters):
    
    def gaussian(x, C, a):    
        return a * np.exp((-(x-wave0) ** 2)/ (2 * sigma ** 2)) + C
    
    def gaussian_noC(x, a):
        return a * np.exp((-(x-wave0) ** 2)/ (2 * sigma ** 2))
    
    fluxerr = np.zeros(iters)
    sigma = 1.3391374247333334
    
    for i in range(len(fluxerr)):
        noiseflux = inputflux + np.random.normal(0, gnoise, len(inputflux))
        lowb = wave0 - 10
        highb = wave0 + 5   
        low = np.where(inwave > lowb)[0][0]
        up = np.where(inwave > highb)[0][0]
        waves = inwave[low:up]
        fluxes = noiseflux[low:up]
        fluxaves = np.mean(fluxes)
        p0list = np.array([fluxaves, 500])
        
        try:
            param, paramcov = curve_fit(gaussian, waves, fluxes, p0list)
            
            flux = quad(gaussian_noC, wave0 - 10 * sigma, wave0 + 10 * sigma, args=(param[1]))[0]
        
            fluxerr[i] = flux
        except:
            fluxerr[i] = np.nan
            
    err = np.array([fluxerr[i] for i in range(len(fluxerr)) if np.isnan(fluxerr[i]) == False])
    err = np.std(err)
    
    return err

### [OII]7319 Refit Function

In [ ]:
def refit7319(inwave, influx, snval, phdata, galvel, mcit, mwebv):
    
    def gaussian(x, C, a):    #a:amplitude, x0:average wavelength, delwave:spectral resolution, C:zero offeset
        return a * np.exp((-(x-w0) ** 2)/ (2 * delwave ** 2)) + C
    
    def gaussian_noC(x, a):
        return a * np.exp((-(x-w0) ** 2)/ (2 * delwave ** 2))

    delwave = 1.3391374247333334
    outflux = np.zeros(len(influx))
    outfluxerr = np.zeros(len(influx))
    wavelength = 7320
    for i in range(len(influx)):
        w0 = wavelength*(phdata[i]['HA6562_VEL']+galvel)/(299792) + wavelength
        lowb = w0 - 10
        highb = w0 + 5        #plus five to not include [O II]7330 peak
        low = np.where(inwave > lowb)[0][0]
        up = np.where(inwave > highb)[0][0]
        waves = inwave[low:up]
        fluxes = influx[i][low:up]
        fluxaves = np.mean(fluxes)
        p0list = np.array([fluxaves, 500])
        
        try:
            param, paramcov = curve_fit(gaussian, waves, fluxes, p0list)
        except:
            outflux[i] = np.nan
            
        flux = quad(gaussian_noC, w0 - 10 * delwave, w0 + 10 * delwave, args=(param[1]))[0]
        signal = param[1]
        noise1 = np.std(influx[i][np.where(inwave > w0 - 30)[0][0]:np.where(inwave > w0 - 10)[0][0]])
        noise2 = np.std(influx[i][np.where(inwave > w0 - 50)[0][0]:np.where(inwave > w0 - 43)[0][0]])
        noise3 = np.std(influx[i][np.where(inwave > w0 + 20)[0][0]:np.where(inwave > w0 + 50)[0][0]])
        noise4 = np.std(influx[i][np.where(inwave > w0 + 65)[0][0]:np.where(inwave > w0 + 85)[0][0]])
        noise5 = np.std(influx[i][np.where(inwave > w0 - 135)[0][0]:np.where(inwave > w0 - 95)[0][0]])
        noise = np.mean([noise1, noise2, noise3, noise4, noise5])
        
        if signal/noise > snval and flux > 0: 
            outflux[i] = flux
            outfluxerr[i] = mcerr7319(inwave, influx[i], noise, w0, mcit)
        else: 
            outflux[i] = np.nan
            outfluxerr[i] = np.nan
    
    R_V = 3.1
    mwcorr7319 = remove(extinction.odonnell94(np.array([7319.990]), mwebv*R_V, R_V), outflux)
    mwcorr7319err = remove(extinction.odonnell94(np.array([7319.990]), mwebv*R_V, R_V), outfluxerr)

    return mwcorr7319, mwcorr7319err

### [OII]7330 Error Function

In [ ]:
def mcerr7330(inwave, inputflux, gnoise, wave0, iters):
    
    def gaussian(x, C, a):    
        return a * np.exp((-(x-wave0) ** 2)/ (2 * sigma ** 2)) + C
    
    def gaussian_noC(x, a):
        return a * np.exp((-(x-wave0) ** 2)/ (2 * sigma ** 2))
    
    fluxerr = np.zeros(iters)
    wave = 7330
    dopv = 126.47 - 0.00978*wave
    sigma = (wave * dopv * 10**13) / (3 * 10**8 * 10**10)
    
    for i in range(len(fluxerr)):
        noiseflux = inputflux + np.random.normal(0, gnoise, len(inputflux))
        lowb = wave0 - 5
        highb = wave0 + 10   
        low = np.where(inwave > lowb)[0][0]
        up = np.where(inwave > highb)[0][0]
        waves = inwave[low:up]
        fluxes = noiseflux[low:up]
        fluxaves = np.mean(fluxes)
        p0list = np.array([fluxaves, 500])
        
        try:
            param, paramcov = curve_fit(gaussian, waves, fluxes, p0list)
            
            flux = quad(gaussian_noC, wave0 - 10 * sigma, wave0 + 10 * sigma, args=(param[1]))[0]
        
            fluxerr[i] = flux
        except:
            fluxerr[i] = np.nan
            
    err = np.array([fluxerr[i] for i in range(len(fluxerr)) if np.isnan(fluxerr[i]) == False])
    err = np.std(err)
    
    return err

### [OII]7330 Refit Function

In [ ]:
def refit7330(inwave, influx, snval, indata, galvel, mcit, mwebv):
    outflux = np.zeros(len(influx))
    outfluxerr = np.zeros(len(influx))
    wave = 7330
    dopv = 126.47 - 0.00978*wave
    d7330 = (wave * dopv * 10**13) / (3 * 10**8 * 10**10)
    
    def gaussian(x, a, C):    #a:amplitude, wavelength:feature wavelength, d7330:spectral resolution, C:zero offeset
        return a * np.exp((-(x-w0) ** 2)/ (2 * d7330 ** 2)) + C
    
    def gaussian_noC(x, a): 
        return a * np.exp((-(x-w0) ** 2)/ (2 * d7330 ** 2))
    
    for i in range(len(influx)):
        w0 = wave*(indata[i]['HA6562_VEL']+galvel)/(299792) + wave
        lowb = w0 - 5
        highb = w0 + 10
        waves = inwave[np.where(inwave > lowb)[0][0]:np.where(inwave > highb)[0][0]]
        fluxes = influx[i][np.where(inwave > lowb)[0][0]:np.where(inwave > highb)[0][0]]
        fluxave = np.mean(influx[i][np.where(inwave > w0 - 40)[0][0]:np.where(inwave > w0 - 20)[0][0]])
        p0list = np.array([500, fluxave])
        
        try:
            param, paramcov = curve_fit(gaussian, waves, fluxes, p0list)
        except:
            outflux[i] = np.nan
            
        flux = quad(gaussian_noC, w0 - 10 * d7330, w0 + 10 * d7330, args=(param[0]))[0]
        sig = param[0]
        noise1 = np.std(influx[i][np.where(inwave > w0 - 40)[0][0]:np.where(inwave > w0 - 20)[0][0]])
        noise2 = np.std(influx[i][np.where(inwave > w0 - 60)[0][0]:np.where(inwave > w0 - 50)[0][0]])
        noise3 = np.std(influx[i][np.where(inwave > w0 + 10)[0][0]:np.where(inwave > w0 + 40)[0][0]])
        noise4 = np.std(influx[i][np.where(inwave > w0 + 60)[0][0]:np.where(inwave > w0 + 72)[0][0]])
        noise5 = np.std(influx[i][np.where(inwave > w0 - 150)[0][0]:np.where(inwave > w0 - 105)[0][0]])
        noise = np.mean([noise1, noise2, noise3, noise4, noise5])
        if sig/noise > snval and flux > 0:
            outflux[i] = flux
            outfluxerr[i] = mcerr7330(inwave, influx[i], noise, w0, mcit)
        else: 
            outflux[i] = np.nan
            outfluxerr[i] = np.nan
    
    R_V = 3.1
    mwcorr7330 = remove(extinction.odonnell94(np.array([7330.730]), mwebv*R_V, R_V), outflux)
    mwcorr7330err = remove(extinction.odonnell94(np.array([7330.730]), mwebv*R_V, R_V), outfluxerr)

    return mwcorr7330, mwcorr7330err

### [NII]5754 Error Function

In [ ]:
def mcerr5755(inwave, inputflux, gnoise, wave0, iters):
    
    def gaussian(x, C, a):    
        return a * np.exp((-(x-wave0) ** 2)/ (2 * sigma ** 2)) + C
    
    def gaussian_noC(x, a):
        return a * np.exp((-(x-wave0) ** 2)/ (2 * sigma ** 2))
    
    fluxerr = np.zeros(iters)
    wave = 5755
    dopv = 126.47 - 0.00978*wave
    sigma = (wave * dopv * 10**13) / (3 * 10**8 * 10**10)
    
    for i in range(len(fluxerr)):
        noiseflux = inputflux + np.random.normal(0, gnoise, len(inputflux))
        lowb = wave0 - 10
        highb = wave0 + 2*sigma   
        low = np.where(inwave > lowb)[0][0]
        up = np.where(inwave > highb)[0][0]
        waves = inwave[low:up]
        fluxes = noiseflux[low:up]
        fluxaves = np.mean(fluxes)
        p0list = np.array([fluxaves, 500])
        
        try:
            param, paramcov = curve_fit(gaussian, waves, fluxes, p0list)
            
            flux = quad(gaussian_noC, wave0 - 10 * sigma, wave0 + 10 * sigma, args=(param[1]))[0]
        
            fluxerr[i] = flux
        except:
            fluxerr[i] = np.nan
            
    err = np.array([fluxerr[i] for i in range(len(fluxerr)) if np.isnan(fluxerr[i]) == False])
    err = np.std(err)
    
    return err

### [NII]5754 Refit

In [ ]:
def refit5755(inwave, influx,snval, phdata, galvel, mcit, mwebv):
    
    def gaussian(x, C, a):    #a:amplitude, x0:average wavelength, delwave:spectral resolution, C:zero offeset
        return a * np.exp((-(x-w0) ** 2)/ (2 * delwave ** 2)) + C
    
    def gaussian_noC(x, a):
        return a * np.exp((-(x-w0) ** 2)/ (2 * delwave ** 2))
    
    wavelength = 5755

    dopv = 126.47 - 0.00978*wavelength
    delwave = (wavelength * dopv * 10**13) / (3 * 10**8 * 10**10)
    outflux = np.zeros(len(influx))
    outfluxerr = np.zeros(len(influx))
    for i in range(len(influx)):
        w0 = wavelength*(phdata[i]['HA6562_VEL']+galvel)/(299792) + wavelength
        lowb = w0 - 10
        highb = w0 + 2*delwave
        low = np.where(inwave > lowb)[0][0]
        up = np.where(inwave > highb)[0][0]
        waves = inwave[low:up]
        fluxes = influx[i][low:up]
        fluxaves = np.mean(influx[i][np.where(inwave > w0 + 180)[0][0]:np.where(inwave > w0 + 250)[0][0]])
        p0list = np.array([fluxaves, 500])
        
        try:
            param, paramcov = curve_fit(gaussian, waves, fluxes, p0list)
        except:
            outflux[i] = np.nan
            
        flux = quad(gaussian_noC, w0 - 10 * delwave, w0 + 10 * delwave, args=(param[1]))[0]
        signal = param[1]
        noise1 = np.std(influx[i][np.where(inwave > w0 - 55)[0][0]:np.where(inwave > w0 - 45)[0][0]])
        noise2 = np.std(influx[i][np.where(inwave > w0 - 25)[0][0]:np.where(inwave > w0 - 8)[0][0]])
        noise3 = np.std(influx[i][np.where(inwave > w0 - 110)[0][0]:np.where(inwave > w0 - 70)[0][0]])
        noise4 = np.std(influx[i][np.where(inwave > w0 + 180)[0][0]:np.where(inwave > w0 + 250)[0][0]])
        noise = np.mean([noise1, noise2, noise3, noise4]) 
        
        if signal/noise > snval and flux > 0:  
            outflux[i] = flux
            outfluxerr[i] = mcerr5755(inwave, influx[i], noise, w0, mcit)
        else: 
            outflux[i] = np.nan
            outfluxerr[i] = np.nan
    
    R_V = 3.1
    mwcorr5755 = remove(extinction.odonnell94(np.array([5754.590]), mwebv*R_V, R_V), outflux)
    mwcorr5755err = remove(extinction.odonnell94(np.array([5754.590]), mwebv*R_V, R_V), outfluxerr)
    
    return mwcorr5755, mwcorr5755err

### [SIII]6312 Error Function

In [ ]:
def mcerr6312(inwave, inputflux, gnoise, wave0, iters):
    
    def gaussian(x, a, C):    #a:amplitude, wave0:feature wavelength, sigma:spectral resolution, C:zero offeset
        return a * np.exp((-(x-wave0) ** 2)/ (2 * sigma ** 2)) + C
    
    def gaussian_noC(x, a): 
        return a * np.exp((-(x-wave0) ** 2)/ (2 * sigma ** 2))
    
    fluxerr = np.zeros(iters)
    wave = 6312
    dopv = 126.47 - 0.00978*wave
    sigma = (wave * dopv * 10**13) / (3 * 10**8 * 10**10)
    
    for i in range(len(fluxerr)):
        noiseflux = inputflux + np.random.normal(0, gnoise, len(inputflux))
        lowb = wave0 - 3*sigma
        highb = wave0 + 12  
        low = np.where(inwave > lowb)[0][0]
        up = np.where(inwave > highb)[0][0]
        waves = inwave[low:up]
        fluxes = noiseflux[low:up]
        fluxaves = np.mean(fluxes)
        p0list = np.array([500, fluxaves])
        
        try:
            param, paramcov = curve_fit(gaussian, waves, fluxes, p0list)
            
            flux = quad(gaussian_noC, wave0 - 10 * sigma, wave0 + 10 * sigma, args=(param[0]))[0]
        
            fluxerr[i] = flux
        except:
            fluxerr[i] = np.nan
            
    err = np.array([fluxerr[i] for i in range(len(fluxerr)) if np.isnan(fluxerr[i]) == False])
    err = np.std(err)
    
    return err

### [SIII]6312 Refit

In [ ]:
def refit6312(inwave, influx, snval, phdata, galvel, mcit, mwebv):
    
    def gaussian(x, a, C, red):    #a:amplitude, wavelength:feature wavelength, d7330:spectral resolution, C:zero offeset
        return a * np.exp((-(x-red) ** 2)/ (2 * delwave ** 2)) + C
    
    def gaussian_noC(x, a, red): 
        return a * np.exp((-(x-red) ** 2)/ (2 * delwave ** 2))
    
    wavelength = 6312

    dopv = 126.47 - 0.00978*wavelength
    delwave = (wavelength * dopv * 10**13) / (3 * 10**8 * 10**10)
    outflux = np.zeros(len(influx))
    outfluxerr = np.zeros(len(influx))
    for i in range(len(influx)):
        w0 = wavelength*(data[i]['HA6562_VEL']+galvel)/(299792) + wavelength
        lowb = w0 - 3*delwave
        highb = w0 + 12
        low = np.where(inwave > lowb)[0][0]
        up = np.where(inwave > highb)[0][0]
        waves = inwave[low:up]
        fluxes = influx[i][low:up]
        fluxaves = np.mean(fluxes)
        p0list = np.array([500, fluxaves, w0])
        
        try:
            param, paramcov = curve_fit(gaussian, waves, fluxes, p0list)
        except:
            outflux[i] = np.nan

        flux = quad(gaussian_noC, w0 - 10 * delwave, w0 + 10 * delwave, args=(param[0], param[2]))[0]
        signal = param[0]
        noise1 = np.std(influx[i][np.where(inwave > w0 - 275)[0][0]:np.where(inwave > w0 - 180)[0][0]])
        noise2 = np.std(influx[i][np.where(inwave > w0 + 63)[0][0]:np.where(inwave > w0 + 90)[0][0]])
        noise3 = np.std(influx[i][np.where(inwave > w0 + 7)[0][0]:np.where(inwave > w0 + 45)[0][0]])
        noise = np.mean([noise1, noise2, noise3])
            
        if signal/noise > snval and flux > 0 and flux < 10**6: 
            outflux[i] = flux
            outfluxerr[i] = mcerr6312(inwave, influx[i], noise, w0, mcit)
        else: 
            outflux[i] = np.nan
            outfluxerr[i] = np.nan
    
    R_V = 3.1
    mwcorr6312 = remove(extinction.odonnell94(np.array([6312.060]), mwebv*R_V, R_V), outflux)
    mwcorr6312err = remove(extinction.odonnell94(np.array([6312.060]), mwebv*R_V, R_V), outfluxerr)
    
    return mwcorr6312, mwcorr6312err

### [OII]3726,3729 MC error function

In [ ]:
def mcerr3727(inputflux, sitcube, gnoise, v_gvar, iters):
    
    flux3727err = np.zeros(iters)
    fitshape = 'sinc'
    #OII3726 = Lines().get_line_cm1('[OII]3726')
    OII3729 = Lines().get_line_cm1('[OII]3729')
    
    for i in range(iters):
        spec = inputflux + np.random.normal(0, gnoise, len(inputflux))
        
        try:
            fit = fit_lines_in_spectrum(spec, [OII3729], step=sitcube.params.step, order=sitcube.params.order, nm_laser=sitcube.params.nm_laser, theta=corr2theta(sitcube.params.axis_corr), zpd_index=sitcube.params.zpd_index, 
                                        wavenumber=True, apodization=1, fmodel=fitshape, pos_def=['1'], pos_cov=[v_gvar], amp_def=['1'], amp_guess=[1],)
            
            flux3727err[i] = fit['flux']
        except:
            flux3727err[i] = np.nan

    err3727 = np.array([flux3727err[i] for i in range(len(flux3727err)) if ~np.isnan(flux3727err[i])])
    err3727 = np.std(err3727)
    
    return err3727

### [OII]3727 Flux fit (using ORCS)

In [ ]:
def fit_3727(phdata, inspec, inwave, incube, ingalvel, inebv, insnval, mcit):

    out_fluxes = np.zeros(len(phdata))
    out_fluxes_err = np.zeros(len(phdata))

    broadening_gvar = gvar.gvar(1, 20)

    S2 = pn.Atom('S', 2)
    O2 = pn.Atom('O', 2)
    temp = 10**4
    fitshape = 'sinc'
    OII3726 = Lines().get_line_cm1('[OII]3726')
    OII3729 = Lines().get_line_cm1('[OII]3729')
    for i in range(len(phdata)):
        
        inspectrum = inspec[i]
        inspectrum[np.isnan(inspectrum)] = 0

        wave3729 = 3728.815
        c = 299792
        red3729 = 1/((wave3729*(phdata[i]['HA6562_VEL']+galvel)/(c) + wave3729) * 10**-8)

        n1 = [np.where(inwave > incube.params.filter_range[0])[0][0],np.where(inwave > red3729 - 175)[0][0]]
        n2 = [np.where(inwave > red3729 + 175)[0][0], np.where(inwave > incube.params.filter_range[1])[0][0]]
        noise = (np.mean(inspectrum[n1[0]:n1[1]]) + np.mean(inspectrum[n2[0]:n2[1]])) / 2
        noisestd = (np.std(inspectrum[n1[0]:n1[1]]) + np.std(inspectrum[n2[0]:n2[1]])) / 2
        inspectrum = (inspectrum) - (noise)

        invel = phdata[i]['HA6562_VEL']+ingalvel 
        velocity_gvar = gvar.gvar(invel, 3)
        
        try:
            fit = fit_lines_in_spectrum(inspectrum, [OII3729], 
                      step=incube.params.step, order=incube.params.order, nm_laser=incube.params.nm_laser, theta=corr2theta(incube.params.axis_corr), zpd_index=incube.params.zpd_index, 
                      wavenumber=True,
                      apodization=1, 
                      fmodel=fitshape,
                      pos_def=['1'],
                      pos_cov=[velocity_gvar],
                      amp_def=['1'], 
                      amp_guess=[1],
                      )
            
            snr = fit['lines_params'][0][1] / noisestd

            if snr > insnval:
                flux = float(fit['flux'][0])
                redline = 1/(float(fit['lines_params'][0][2]) * 10**-8)

                R_V = 3.1
                mwcorr = remove(extinction.odonnell94(np.array([redline]), inebv*R_V, R_V), flux)

                out_fluxes[i] = mwcorr[0]
                out_fluxes_err[i] = mcerr3727(inspectrum, incube, noisestd, velocity_gvar, mcit)
            else:
                out_fluxes[i] = np.nan
                out_fluxes_err[i] = np.nan
        except:
            out_fluxes[i] = np.nan
            out_fluxes_err[i] = np.nan
            continue
    
    return out_fluxes, out_fluxes_err

### Multiprocessing of the refit functions

In [ ]:
if __name__ == "__main__":

    pronum = 16 

    split = pronum/5
    batch = math.ceil(len(data)/split)
    fluxlist = [fluxin[i:i+batch] for i in range(0, len(data), batch)]
    speclist = [sitspec[i:i+batch] for i in range(0, len(data), batch)]
    datalist = [data[i:i+batch] for i in range(0, len(data), batch)]

    paramlist = [(lam, fluxlist[i], sn, datalist[i], galvel, mciters, ebv) for i in range(len(datalist))]
    sitparamlist = [(datalist[i], speclist[i], sitlam, cube, galvel, ebv, sn, mciters) for i in range(len(datalist))]
    funcs = [refit5755, refit6312, refit7319, refit7330, fit_3727]
    
    pool = mp.Pool(processes = pronum)          #count processes are inititiated

    list5755 = [pool.apply_async(funcs[0], args = p) for p in paramlist]
    list6312 = [pool.apply_async(funcs[1], args = p) for p in paramlist]
    list7319 = [pool.apply_async(funcs[2], args = p) for p in paramlist]
    list7330 = [pool.apply_async(funcs[3], args = p) for p in paramlist]
    list3727 = [pool.apply_async(funcs[4], args = p) for p in sitparamlist]

results5755 = [list5755[i].get() for i in range(len(list5755))]
results6312 = [list6312[i].get() for i in range(len(list6312))]
results7319 = [list7319[i].get() for i in range(len(list7319))]
results7330 = [list7330[i].get() for i in range(len(list7330))]
results3727 = [list3727[i].get() for i in range(len(list3727))]

In [ ]:
ref5755 = np.concatenate([results5755[i][0] for i in range(len(list5755))])
ref5755err = np.concatenate([results5755[i][1] for i in range(len(list5755))])
ref6312 = np.concatenate([results6312[i][0] for i in range(len(list6312))])
ref6312err = np.concatenate([results6312[i][1] for i in range(len(list6312))])
ref7319 = np.concatenate([results7319[i][0] for i in range(len(list7319))])
ref7319err = np.concatenate([results7319[i][1] for i in range(len(list7319))])
ref7330 = np.concatenate([results7330[i][0] for i in range(len(list7330))])
ref7330err = np.concatenate([results7330[i][1] for i in range(len(list7330))])
ref3727 = np.concatenate([results3727[i][0] for i in range(len(list3727))])
ref3727err = np.concatenate([results3727[i][1] for i in range(len(list3727))])

print(f'{len(np.where(~np.isnan(ref5755))[0])} [NII]5755 flux measurements, {len(np.where(~np.isnan(ref6312))[0])} [SIII]6312 flux measurements, {len(np.where(~np.isnan(ref7319))[0])} [OII]7319 flux measurements, {len(np.where(~np.isnan(ref7330))[0])} [OII]7330 flux measurements, {len(np.where(~np.isnan(ref3727))[0])} [OII]3727 flux measurements')

### Compile Refits

In [ ]:
data.add_columns([ref5755, ref6312, ref7319, ref7330, ref5755err, ref6312err, ref7319err, ref7330err, ref3727, ref3727err], 
                 names=('NII5754_FLUX_REFIT','SIII6312_FLUX_REFIT','OII7319_FLUX_REFIT', 'OII7330_FLUX_REFIT',
                       'NII5754_FLUX_REFIT_ERR','SIII6312_FLUX_REFIT_ERR','OII7319_FLUX_REFIT_ERR', 'OII7330_FLUX_REFIT_ERR',
                       'OII3727_FLUX', 'OII3727_FLUX_ERR'))

### Correct for extinction

In [ ]:
def corr(indata):
    
    R_V = 3.1 
    A_V = np.array([indata[i]['EBV'] for i in range(len(indata))]) * R_V

    corr7319 = np.array([remove(extinction.odonnell94(np.array([7319.990]), A_V[i], R_V), indata[i]['OII7319_FLUX_REFIT'])[0] for i in range(len(indata))])
    corr7319err = np.array([remove(extinction.odonnell94(np.array([7319.990]), A_V[i], R_V), indata[i]['OII7319_FLUX_REFIT_ERR'])[0] for i in range(len(indata))])

    corr7330 = np.array([remove(extinction.odonnell94(np.array([7330.730]), A_V[i], R_V), indata[i]['OII7330_FLUX_REFIT'])[0] for i in range(len(indata))])
    corr7330err = np.array([remove(extinction.odonnell94(np.array([7330.730]), A_V[i], R_V), indata[i]['OII7330_FLUX_REFIT_ERR'])[0] for i in range(len(indata))])

    corr5755 = np.array([remove(extinction.odonnell94(np.array([5754.590]), A_V[i], R_V), indata[i]['NII5754_FLUX_REFIT'])[0] for i in range(len(indata))])
    corr5755err = np.array([remove(extinction.odonnell94(np.array([5754.590]), A_V[i], R_V), indata[i]['NII5754_FLUX_REFIT_ERR'])[0] for i in range(len(indata))])
    
    corr6312 = np.array([remove(extinction.odonnell94(np.array([6312.060]), A_V[i], R_V), indata[i]['SIII6312_FLUX_REFIT'])[0] for i in range(len(indata))])
    corr6312err = np.array([remove(extinction.odonnell94(np.array([6312.060]), A_V[i], R_V), indata[i]['SIII6312_FLUX_REFIT_ERR'])[0] for i in range(len(indata))])

    corr3727 = np.array([remove(extinction.odonnell94(np.array([3728.815]), A_V[i], R_V), indata[i]['OII3727_FLUX'])[0] for i in range(len(indata))])
    corr3727err = np.array([remove(extinction.odonnell94(np.array([3728.815]), A_V[i], R_V), indata[i]['OII3727_FLUX_ERR'])[0] for i in range(len(indata))])
    
    indata.add_columns([corr5755, corr6312, corr7319, corr7330, corr5755err, corr6312err, corr7319err, corr7330err, corr3727, corr3727err], 
                       names=('NII5754_FLUX_CORR_REFIT', 'SIII6312_FLUX_CORR_REFIT', 'OII7319_FLUX_CORR_REFIT', 'OII7330_FLUX_CORR_REFIT',
                             'NII5754_FLUX_CORR_REFIT_ERR', 'SIII6312_FLUX_CORR_REFIT_ERR', 'OII7319_FLUX_CORR_REFIT_ERR', 'OII7330_FLUX_CORR_REFIT_ERR',
                             'OII3727_FLUX_CORR', 'OII3727_FLUX_CORR_ERR'))

    return indata

In [ ]:
refitdata = corr(data)

### Save Refits

In [ ]:
refitdata.write(f'/home/habjan/jupfiles/data/{galaxy}_refit+SITELLEfits_data.fits', overwrite=True)  #, overwrite=True